# Lab-10-3-1 Visdom Example (22.07.27)

In [1]:
import torch
import torch.nn as nn

import torchvision
import torchvision.datasets as dsets

## import visdom

![](figs/turn_on_terminal.png)

Jupyter Notebook > Terminal 를 새로 켜서 `python -m visdom.server` 를 입력하세요!

## Colab 에서 Visdom (Pytorch 용 Tensorboard) 사용법

In [26]:
! npm install -g localtunnel
get_ipython().system_raw('/usr/local/bin/python -m pip install visdom')
get_ipython().system_raw('/usr/local/bin/python -m visdom.server -port 6006 >> visdomlog.txt 2>&1 &')
get_ipython().system_raw('lt --port 6006 >> url.txt 2>&1 &')
import time
time.sleep(5)
! cat url.txt
import visdom
time.sleep(5)
vis = visdom.Visdom(port='6006')
print(vis)
time.sleep(3)
vis.text('testing')
! cat visdomlog.txt

/tools/node/bin/lt -> /tools/node/lib/node_modules/localtunnel/bin/lt.js
+ localtunnel@2.0.2
added 22 packages from 22 contributors in 1.855s
your url is: http://fancy-shirts-pump-35-247-106-5.loca.lt


Setting up a new session...


/usr/local/lib/python3.7/dist-packages/visdom/server.py:39: DeprecationWarning: zmq.eventloop.ioloop is deprecated in pyzmq 17. pyzmq now works with default tornado and asyncio eventloops.
  ioloop.install()  # Needs to happen before any tornado imports!
INFO:root:Application Started
INFO:tornado.access:200 POST /env/main (127.0.0.1) 0.58ms
INFO:tornado.access:101 GET /vis_socket (127.0.0.1) 0.40ms
INFO:root:Opened visdom socket from ip: 127.0.0.1
INFO:tornado.access:200 POST /events (127.0.0.1) 0.63ms


In [24]:
import visdom
vis = visdom.Visdom()

Setting up a new session...


## Text

In [27]:
vis.text("Hello, world!",env="main")

'window_3aefa2ba0f50e2'

## image

In [28]:
a=torch.randn(3,200,200)
vis.image(a)

'window_3aefa2bc6d08c8'

## images

In [29]:
vis.images(torch.Tensor(3,3,28,28))

'window_3aefa2bea01712'

## example (using MNIST and CIFAR10)

In [30]:
# 시간이 좀 걸립니다.
MNIST = dsets.MNIST(root="./MNIST_data",train = True,transform=torchvision.transforms.ToTensor(), download=True)
cifar10 = dsets.CIFAR10(root="./cifar10",train = True, transform=torchvision.transforms.ToTensor(),download=True)

Files already downloaded and verified


#### CIFAR10

In [31]:
data = cifar10.__getitem__(0)
print(data[0].shape)
vis.images(data[0],env="main")

torch.Size([3, 32, 32])


'window_3aefa2c6bef0d4'

#### MNIST

In [32]:
data = MNIST.__getitem__(0)
print(data[0].shape)
vis.images(data[0],env="main")

torch.Size([1, 28, 28])


'window_3aefa2cc3bbb72'

#### Check dataset

In [33]:
data_loader = torch.utils.data.DataLoader(dataset = MNIST,
                                          batch_size = 32,
                                          shuffle = False)

In [34]:
for num, value in enumerate(data_loader):
    value = value[0]
    print(value.shape)
    vis.images(value)
    break

torch.Size([32, 1, 28, 28])


In [45]:
vis.close(env="main")

''

## Line Plot

In [36]:
Y_data = torch.randn(5)
plt = vis.line (Y=Y_data)

In [37]:
X_data = torch.Tensor([1,2,3,4,5])
plt = vis.line(Y=Y_data, X=X_data)

### Line update

In [38]:
Y_append = torch.randn(1)
X_append = torch.Tensor([6])

vis.line(Y=Y_append, X=X_append, win=plt, update='append')

'window_3aefa2d8c5982a'

### multiple Line on single windows

In [39]:
num = torch.Tensor(list(range(0,10)))
num = num.view(-1,1)
num = torch.cat((num,num),dim=1)

plt = vis.line(Y=torch.randn(10,2), X = num)

### Line info

In [40]:
plt = vis.line(Y=Y_data, X=X_data, opts = dict(title='Test', showlegend=True))

In [41]:
plt = vis.line(Y=Y_data, X=X_data, opts = dict(title='Test', legend = ['1번'],showlegend=True))

In [42]:
plt = vis.line(Y=torch.randn(10,2), X = num, opts=dict(title='Test', legend=['1번','2번'],showlegend=True))

## make function for update line

In [46]:
def loss_tracker(loss_plot, loss_value, num):
    '''num, loss_value, are Tensor'''
    vis.line(X=num,
             Y=loss_value,
             win = loss_plot,
             update='append'
             )

In [47]:
plt = vis.line(Y=torch.Tensor(1).zero_())

for i in range(500):
    loss = torch.randn(1) + i
    loss_tracker(plt, loss, torch.Tensor([i]))

## close the window

In [48]:
vis.close(env="main")

''